In [20]:
import pandas
import sklearn.preprocessing

pandas.set_option("display.precision", 2)

In [21]:
df_features = pandas.read_feather('features/3d.feather')
df_features.describe()

,0,1,2
count,1046.00,1046.00,1046.00
mean,8.89,5.13,2.81
std,0.02,0.71,1.31
min,8.83,1.38,-1.00
25%,8.88,4.70,2.05
50%,8.89,5.16,2.94
75%,8.90,5.61,3.73
max,8.93,6.95,5.62


In [3]:
features = sklearn.preprocessing.StandardScaler().fit_transform(df_features.to_numpy())
pandas.DataFrame(features, columns = df_features.columns).describe()

,0,1,2
count,1.046000e+03,1.046000e+03,1.046000e+03
mean,-7.317367e-14,8.151542e-16,1.086872e-16
std,1.000478e+00,1.000478e+00,1.000478e+00
min,-3.261186e+00,-5.266736e+00,-2.919209e+00
25%,-5.742665e-01,-6.103810e-01,-5.834045e-01
50%,1.767408e-01,4.670871e-02,1.002772e-01
75%,7.246725e-01,6.685867e-01,6.999822e-01
max,2.236961e+00,2.561917e+00,2.153033e+00


In [4]:
%load_ext autoreload
%autoreload 2

In [10]:
from SDA.SDA import SDA
sda = SDA()
my_result, my_df_st_edges = sda.apply(features, save = ('results/result.csv', 'results/df_st_edged.csv'))

Applying to 1046 samples with 3 features each
Running stage 1


  0%|          | 0/589 [00:00<?, ?it/s]

Running stage 2


  0%|          | 0/672 [00:00<?, ?it/s]

In [23]:
# Plot parameters for each number of stages
min_stage_length = 0 # minimum acceptable stage length
best_results = [ ]

for n_st in range(3, 15 + 2):
    df = my_result[(my_result['N_stages']==n_st) & (my_result['St_len_min'] >= min_stage_length)]
    best_row = df.sort_values(by = 'Avg-Silh').iloc[0]
    best_results.append(best_row)
    
df_st_edges_best = pandas.DataFrame(best_results)
display(df_st_edges_best)

,St_len_min,K_nb_max,N_cl_max,N_stages,Cl_cen,St_edges,Ward_dist,Cen_dist,Silh,Cal-Har,Dav-Bold,Avg-Silh,Avg-Cal-Har,Avg-Dav-Bold
673,20,40,15,3,Mode,"[0, 358, 609, 1046]",170.73,0.92,0.03,98.92,6.21,0.08,77.93,5.06
1684,60,40,15,4,Mode,"[0, 358, 566, 864, 1046]",127.35,0.88,0.02,74.32,6.23,0.08,56.96,4.66
1687,60,40,15,5,Mode,"[0, 94, 358, 566, 864, 1046]",102.88,0.97,-0.05,79.33,5.32,0.09,58.30,3.92
1648,60,40,10,6,Mode,"[0, 94, 238, 358, 609, 864, 1046]",73.44,0.91,-0.07,79.29,5.75,0.09,43.65,2.99
433,0,50,15,7,Mode,"[0, 105, 251, 358, 609, 681, 864, 1046]",65.99,0.89,-0.07,73.04,8.18,0.08,38.86,4.23
268,0,45,10,8,Mode,"[0, 105, 251, 358, 609, 681, 811, 864, 1046]",41.55,0.81,-0.09,63.55,8.12,0.07,27.90,4.43
1447,40,50,15,9,Mode,"[0, 105, 251, 358, 486, 609, 681, 811, 864, 1046]",40.61,0.80,-0.09,57.39,10.27,0.07,26.37,5.67
399,0,50,10,10,Median,"[0, 94, 250, 358, 490, 609, 681, 795, 864, 100...",40.42,0.79,-0.10,52.26,9.78,0.06,25.80,5.54
404,0,50,10,11,Mean,"[0, 53, 100, 244, 360, 491, 589, 687, 798, 870...",33.98,0.77,-0.14,46.11,7.14,0.06,19.90,3.88
1791,60,45,10,12,Median,"[0, 104, 194, 250, 358, 490, 609, 681, 728, 81...",21.04,0.62,-0.13,42.26,18.42,0.06,15.00,10.45


In [24]:
# Optimal number of clusters

n_stages = 10 # expert choise of result number of stages

st_len = df_st_edges_best[df_st_edges_best['N_stages']==n_stages]['St_len_min'].iloc[0]
k_nb = df_st_edges_best[df_st_edges_best['N_stages']==n_stages]['K_nb_max'].iloc[0]
n_cl = df_st_edges_best[df_st_edges_best['N_stages']==n_stages]['N_cl_max'].iloc[0]

# Cluster center type
cl_center_type = df_st_edges_best[df_st_edges_best['N_stages']==n_stages]['Cl_cen'].iloc[0]
print(cl_center_type)

# Stage boundary epoch numbers
st_edges_result = df_st_edges_best[df_st_edges_best['N_stages']==n_stages]['St_edges'].iloc[0]
#st_edges_result = [0, 92, 282, 492, 560, 666, 784, 857, 976, 1046] # results for m10
print(st_edges_result)

Median
[0, 94, 250, 358, 490, 609, 681, 795, 864, 1003, 1046]
